In [1]:
import numpy as np
from PIL import Image

TOLERANCE = 1e-3
MAXDIST = 1e6

In [2]:
def length(x, y, z):
    return np.sqrt(x ** 2 + y ** 2 + z ** 2)

In [3]:
class Sphere:
    def __init__(self, xc, yc, zc, r):
        self.xc = xc
        self.yc = yc
        self.zc = zc
        self.r = r
    def dist(self, x, y, z):
        dx = x - self.xc
        dy = y - self.yc
        dz = z - self.zc
        dl = length(dx, dy, dz)
        return np.maximum(0.0, dl - self.r)
    def norm(self, x, y, z):
        dx = x - self.xc
        dy = y - self.yc
        dz = z - self.zc
        dl = length(dx, dy, dz)
        return dx / dl, dy / dl, dz / dl

class Box:
    def __init__(self, xc, yc, zc, hs):
        self.xc = xc
        self.yc = yc
        self.zc = zc
        self.hs = hs
    def dist(self, x, y, z):
        dx = np.maximum(0.0, np.abs(x - self.xc) - self.hs)
        dy = np.maximum(0.0, np.abs(y - self.yc) - self.hs)
        dz = np.maximum(0.0, np.abs(z - self.zc) - self.hs)
        dl = length(dx, dy, dz)
        dl = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
        return np.maximum(0.0, dl)
    def norm(self, x, y, z):
        dx = x - self.xc
        dy = y - self.yc
        dz = z - self.zc
        ux = np.where(
            np.logical_and(np.abs(dx) > np.abs(dy), np.abs(dx) > np.abs(dz)),
            dx / np.abs(dx), 0.0
        )
        uy = np.where(
            np.logical_and(np.abs(dy) > np.abs(dx), np.abs(dy) > np.abs(dz)),
            dy / np.abs(dy), 0.0
        )
        uz = np.where(
            np.logical_and(np.abs(dz) > np.abs(dx), np.abs(dz) > np.abs(dy)),
            dz / np.abs(dz), 0.0
        )
        return ux, uy, uz

In [4]:
# define geometry

objs = []

# add random spheres
for i in range(3):
    obj = Sphere(
        (np.random.random() - 0.5) * 5,
        (np.random.random() - 0.5) * 5,
        np.random.random() * 4 + 10,
        np.random.random() + 0.5
    )
    objs.append(obj)

# add random boxes
for i in range(3):
    obj = Box(
        (np.random.random() - 0.5) * 5,
        (np.random.random() - 0.5) * 5,
        np.random.random() * 4 + 10,
        np.random.random() + 0.5
    )
    objs.append(obj)

# add something vaguely resembling ground
objs.append(Box(
    1005.0, 0.0, 0.0, 1000.0
))

In [5]:
resolution_width = 400
resolution_height = 300
resolution_points = resolution_width * resolution_height

# current space point for each pixel
x = np.zeros(resolution_points)
y = np.zeros(resolution_points)
z = np.zeros(resolution_points)

# points on camera plane
u = np.linspace(-0.4, 0.4, resolution_width)
v = np.linspace(-0.3, 0.3, resolution_height)
uu, vv = np.meshgrid(v, u)
uu = uu.flatten()
vv = vv.flatten()

# unit vectors in pixel direction
ll = length(1, uu, vv)
dx = uu / ll
dy = vv / ll
dz = 1 / ll

# step a fixed number of times
for i in range(1000):
    l = np.ones(x.size, dtype=float) * MAXDIST
    for obj in objs:
        l_i = obj.dist(x, y, z)
        # where smaller, keep the smaller distance
        l = np.minimum(l, l_i)
    x += l * dx
    y += l * dy
    z += l * dz

In [6]:
# determine closest object

material_index = -1 * np.ones(l.shape, dtype=int)
ux = np.zeros(l.shape, dtype=float)
uy = np.zeros(l.shape, dtype=float)
uz = np.zeros(l.shape, dtype=float)
for i_obj in range(len(objs)):
    l_i = objs[i_obj].dist(x, y, z)
    ux_i, uy_i, uz_i = objs[i_obj].norm(x, y, z)
    where = np.abs(l - l_i) < TOLERANCE
    material_index = np.where(where, i_obj, material_index)
    ux = np.where(where, ux_i, ux)
    uy = np.where(where, uy_i, uy)
    uz = np.where(where, uz_i, uz)

fog = np.power(0.5, length(x, y, z) / 50.0)
fog_color = np.array([200, 200, 255])
sunlight = 0.3 + 0.7 * np.maximum(-ux * 0.5 + -uy * 0.3 + uz * -0.2, 0.0)

def rand_color():
    r = np.random.random()
    g = np.random.random()
    b = np.random.random()
    mx = max(r, g, b)
    return np.array([r / mx * 255, g / mx * 255, b / mx * 255])
colors = [rand_color() for i in range(len(objs))]

r = np.ones(l.shape, dtype=int) * fog_color[0]
g = np.ones(l.shape, dtype=int) * fog_color[1]
b = np.ones(l.shape, dtype=int) * fog_color[2]

for i in range(len(objs)):
    r_i = (fog * colors[i][0] * sunlight + (1.0 - fog) * fog_color[0]).astype(dtype=int)
    g_i = (fog * colors[i][1] * sunlight + (1.0 - fog) * fog_color[1]).astype(dtype=int)
    b_i = (fog * colors[i][2] * sunlight + (1.0 - fog) * fog_color[2]).astype(dtype=int)
    where = (i == material_index)
    r = np.where(where, r_i, r)
    g = np.where(where, g_i, g)
    b = np.where(where, b_i, b)

In [7]:
img = Image.new(mode = 'RGB', size = (resolution_width, resolution_height))
pix = img.load()
for i in range(resolution_width):
    for j in range(resolution_height):
        idx = i * resolution_height + j
        pix[i, j] = (r[idx], g[idx], b[idx])
img.save('out.png')